In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import verticapy as vp
from IPython.display import display,clear_output, IFrame, HTML
import pickle
import webbrowser
from ipyfilechooser import FileChooser
import verticapy as vp
from verticapy.performance.vertica import QueryProfilerInterface
vp.set_option("print_info", False)

In [2]:
def reset_values(query_val = None, key_val = None, target_schema_val = None, transaction_val = None, statement_val = None, file_val = None):    
    if query_val:
        %store query_val
    else:
        %store -d query_val
    if key_val:
        %store key_val
    else: 
        %store -d key_val
    if target_schema_val:
        %store target_schema_val
    else:
        %store -d target_schema_val
    if transaction_val:
        %store transaction_val
    else:
        %store -d transaction_val
    if statement_val:
        %store statement_val
    else:
        %store -d statement_val
    if file_val:
        %store file_val
    else:
        %store -d file_val
    notebook_url = '/voila/render/ui/qprof-ui.ipynb'
    display(HTML(f"<script>window.open('{notebook_url}');window.close();</script>"))

In [3]:
%%html
<h2> Connection Info </h2>

In [4]:
print("\033[1mDatabase\033[0m:", vp.current_connection().options['database'])
print("\033[1mUsername\033[0m:", vp.current_connection().options['user'])

Database: vdash
Username: sfduty_rw


____

# Load

In [8]:

button_2 = widgets.Button(description="Get Query Plan")
output_2 = widgets.Output()

key_val = None
target_schema_val = None


def on_button_clicked_2(b):
    global key_val, target_schema_val
    output_2.clear_output(wait=True)
    with output_2:
        key_val = key.value
        target_schema_val = target_schema.value
        reset_values(key_val = key_val, target_schema_val = target_schema_val)

button_2.on_click(on_button_clicked_2)


button_3 = widgets.Button(description="Get Query Plan")
file_val = None
output_3 = widgets.Output()

def on_button_clicked_3(b):
    global key_val, target_schema_val, file_val
    output_3.clear_output(wait=True)
    with output_3:
        key_val = key.value
        target_schema_val = target_schema.value
        file_val = uploader.value
        if key_val == "" or target_schema_val == "" or file_val == "":
            print("Error! You must provide value for all: Schema, Key and File.")
            return
        else:
            reset_values(key_val = key_val, target_schema_val = target_schema_val, file_val = file_val)

button_3.on_click(on_button_clicked_3)

In [6]:
key = widgets.Text(description = "Key", placeholder = "Enter Key", layout=widgets.Layout(width='300px'))
key.style.description_width = '100px'
target_schema = widgets.Text(description = "Target Schema", placeholder = "Enter Schema",layout=widgets.Layout(width='300px'))
target_schema.style.description_width = '100px'
uploader = FileChooser('')

from ipyfilechooser import FileChooser


tab = widgets.Tab()
child_1 = widgets.VBox([target_schema,key,button_2, output_2])
child_2 = widgets.VBox([target_schema,key,uploader,button_3, output_3])
tab.children = [child_1, child_2]
tab.titles = ["From database", "From a file"]
display(tab)

NameError: name 'button_2' is not defined

_____

# Create

In [20]:
button_01 = widgets.Button(description="Query Request Table", layout=widgets.Layout(width='auto'))
output_01 = widgets.Output()
display(button_01, output_01)

def on_button_clicked_01(b):
    output_01.clear_output(wait=True)
    with output_01:
        sql_query = "SELECT * from query_requests WHERE request_type = 'QUERY' AND request LIKE 'select%' ORDER BY request_duration_ms DESC LIMIT 6;"
        result = vp.vDataFrame(sql_query)
        display(HTML(result._repr_html_()))
        

button_01.on_click(on_button_clicked_01)

Button(description='Query Request Table', layout=Layout(width='auto'), style=ButtonStyle())

Output()

In [21]:
transaction = widgets.Text(description = "Transaction ID", placeholder = "Enter a Transaction ID", layout=widgets.Layout(width='300px'))
transaction.style.description_width = '100px'
statement = widgets.Text(description = "Statement ID", placeholder = "Enter a Statement ID", layout=widgets.Layout(width='300px'))
statement.style.description_width = '100px'


In [22]:
button_1 = widgets.Button(description="Get Query Plan")
output_1 = widgets.Output()
transaction_val = None
statement_val = None

def on_button_clicked_1(b):
    output_1.clear_output(wait=True)
    with output_1:
        transaction_val = transaction.value
        statement_val = statement.value
        reset_values(statement_val = statement_val, transaction_val = transaction_val)

button_1.on_click(on_button_clicked_1)
create_0 = widgets.VBox([transaction,statement, button_1, output_1])

In [23]:
query = widgets.Textarea(
    placeholder='Enter a SQL Query',
    description='SQL Query:',
    disabled=False,
    layout=widgets.Layout(width='700px', height = '300px')
)

In [24]:
button = widgets.Button(description="Get Query Plan")
output_0 = widgets.Output()
query_val = None

def on_button_clicked_0(b):
    output_0.clear_output(wait=True)
    global query_val
    with output_0:
        query_val = query.value
        reset_values(query_val = query_val)

button.on_click(on_button_clicked_0)
create_1 = widgets.VBox([query, button, output_0])


In [25]:
tab_2 = widgets.Tab()
tab_2.children = [create_0, create_1]
tab_2.titles = ["Transaction & Statement", "From SQL"]
display(tab_2)

_____

In [26]:
from IPython.display import HTML

display(
    HTML(
        """
<style>
.jupyter-widgets.widget-tab > .lm-TabBar .lm-TabBar-tab {
    flex: 0 1 auto
}
</style>
"""
    )
)